In [5]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import talib as ta # TA-Lib for calculation of indicators
from library.technicalIndicatorsCalculator import TechnicalIndicatorsCalculator
from library.datalabeler import DataLabeler
from library.dataadjuster import DataAdjuster
import os

#### **Configuration**

In [6]:
finance = "XLE"

In [7]:
# Define la ruta raíz desde donde comenzar a recorrer las carpetas
root_path = f'C:/Users/Usuario/Desktop/Maestria/FinTech/CNN2/data/{finance}'

#### **Multiple data**

In [12]:
for root, dirs, files in os.walk(f'{root_path}/dataset'):
    # Imprimir los nombres de todas las carpetas en el directorio actual
    dirs[:] = [d for d in dirs if d not in ('training', 'testing')]
    for dir_name in dirs:
        # Load data
        training_data = pd.read_csv(f'{os.path.join(root, dir_name)}/training.csv')
        testing_data  = pd.read_csv(f'{os.path.join(root, dir_name)}/testing.csv')

        training_data.dropna(inplace = True)
        testing_data.dropna(inplace = True)
        
        #training_data.set_index("Date",inplace = True)
        #testing_data.set_index("Date",inplace = True)

        # Normalize training data
        adjuster_training = DataAdjuster(training_data)
        adjusted_data_training = adjuster_training.adjust_data()
        adjusted_data_training.drop(labels = ["Open", "Adj Close", "AdjustRatio", "Low","High", "Close"], axis = 1, inplace = True) # Drop Volume column

        # Normalize testing data
        adjuster_testing = DataAdjuster(testing_data)
        adjusted_data_testing = adjuster_testing.adjust_data()
        adjusted_data_testing.drop(labels = ["Open", "Adj Close", "AdjustRatio", "Low","High", "Close"], axis = 1, inplace = True) # Drop Volume column

        #Calculate Technical Indicators - Training
        calc_training = TechnicalIndicatorsCalculator(adjusted_data_training)
        calc_training.calculate_all_indicators()
        adjusted_data_training.dropna(inplace = True)
        adjusted_data_training.drop(labels = ["Volume", "AdjustedLow", "AdjustedHigh"], axis = 1, inplace = True) # Drop Volume column
        adjusted_data_training.rename(columns = {"AdjustedClose":"Close", "AdjustedOpen":"Open"}, inplace = True)
        df_training = adjusted_data_training[['Open','Close']].copy()
        df_training.reset_index(drop=False, inplace=True)

        #Calculate Technical Indicators - Testing
        calc_testing = TechnicalIndicatorsCalculator(adjusted_data_testing)
        calc_testing.calculate_all_indicators()
        adjusted_data_testing.dropna(inplace = True)
        adjusted_data_testing.drop(labels = ["Volume", "AdjustedLow", "AdjustedHigh"], axis = 1, inplace = True) # Drop Volume column
        adjusted_data_testing.rename(columns = {"AdjustedClose":"Close", "AdjustedOpen":"Open"}, inplace = True)
        df_testing = adjusted_data_testing[['Open','Close']].copy()
        df_testing.reset_index(drop=False, inplace=True)

        # Implement labelling for the output - Training
        labeler_training = DataLabeler(df_training, window_size=11)
        labeler_training.label_data()
        labeled_data_training = labeler_training.get_labeled_data()
        labeled_data_training = labeled_data_training[['Label', 'Open', 'Close']]
        labeled_data_training.insert(0, 'Date', list(adjusted_data_training['Date']))
        adjusted_data_training.drop(labels = ["Open","Close"], axis = 1, inplace = True) 

        # Implement labelling for the output - Testing
        labeler_testing = DataLabeler(df_testing, window_size=11)
        labeler_testing.label_data()
        labeled_data_testing = labeler_testing.get_labeled_data()
        labeled_data_testing = labeled_data_testing[['Label', 'Open', 'Close']]
        labeled_data_testing.insert(0, 'Date', list(adjusted_data_testing['Date']))
        adjusted_data_testing.drop(labels = ["Open","Close"], axis = 1, inplace = True) 

        scaler = MinMaxScaler(feature_range=(-1, 1))

        # Scale all data - Training
        adjusted_data_training.drop(columns='Date', inplace=True)
        columns = np.array(adjusted_data_training.columns)
        scaled_data_training = pd.DataFrame(scaler.fit_transform(adjusted_data_training), columns=columns)
        scaled_data_training.insert(0, 'Close', list(labeled_data_training['Close']))
        scaled_data_training.insert(0, 'Open', list(labeled_data_training['Open']))
        scaled_data_training.insert(0, 'Label', list(labeled_data_training['Label']))
        scaled_data_training.insert(0, 'Date', list(labeled_data_training['Date']))

        #  Scale all data - Testing
        adjusted_data_testing.drop(columns='Date', inplace=True)
        columns = np.array(adjusted_data_testing.columns)
        scaled_data_testing = pd.DataFrame(scaler.fit_transform(adjusted_data_testing), columns=columns)
        scaled_data_testing.insert(0, 'Close', list(labeled_data_testing['Close']))
        scaled_data_testing.insert(0, 'Open', list(labeled_data_testing['Open']))
        scaled_data_testing.insert(0, 'Label', list(labeled_data_testing['Label']))
        scaled_data_testing.insert(0, 'Date', list(labeled_data_testing['Date']))

        scaled_data_training.set_index("Date",inplace = True)
        scaled_data_testing.set_index("Date",inplace = True)
        
        # Save data
        scaled_data_training.to_csv(f'{os.path.join(root, dir_name)}/data_training_transformed.csv')
        scaled_data_testing.to_csv(f'{os.path.join(root, dir_name)}/data_testing_transformed.csv')

c:\Users\Usuario\Desktop\Maestria\FinTech\CNN2\library\technicalIndicatorsCalculator.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.data[f'TEMA_{i}'] = ta.TEMA(self.data["AdjustedClose"], timeperiod=i+1)
c:\Users\Usuario\Desktop\Maestria\FinTech\CNN2\library\technicalIndicatorsCalculator.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.data[f'TEMA_{i}'] = ta.TEMA(self.data["AdjustedClose"], timeperiod=i+1)
c:\Users\Usuario\Desktop\Maestria\FinTech\CNN2\library\technicalIndicatorsCalculator.py:47: PerformanceW

#### **Single data**

In [45]:
# Load data
data = pd.read_csv(f'{root_path}/data.csv')
data.dropna(inplace = True)
#data.set_index("Date",inplace = True)

# Normalize data
adjuster = DataAdjuster(data)
adjusted_data = adjuster.adjust_data()
adjusted_data.drop(labels = ["Open", "Adj Close", "AdjustRatio", "Low","High", "Close"], axis = 1, inplace = True) # Drop Volume column

#Calculate Technical Indicators
calc = TechnicalIndicatorsCalculator(adjusted_data)
calc.calculate_all_indicators()
adjusted_data.dropna(inplace = True)
adjusted_data.drop(labels = ["Volume", "AdjustedOpen", "AdjustedLow", "AdjustedHigh"], axis = 1, inplace = True) # Drop Volume column
adjusted_data.rename(columns = {"AdjustedClose":"Close"}, inplace = True)
df = adjusted_data[['Close']].copy()
df.reset_index(drop=False, inplace=True)

# Implement labelling for the output
labeler = DataLabeler(df, window_size=11)
labeler.label_data()
labeled_data = labeler.get_labeled_data()
labeled_data = labeled_data[['Label', 'Close']]
adjusted_data.drop(labels = ["Close"], axis = 1, inplace = True) 

# Scale all data
adjusted_data.drop(columns='Date', inplace=True)
columns = np.array(adjusted_data.columns)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = pd.DataFrame(scaler.fit_transform(adjusted_data), columns=columns)
scaled_data.insert(0, 'Close', list(labeled_data['Close']))
scaled_data.insert(0, 'Label', list(labeled_data['Label']))

# Save data
scaled_data.to_csv(f'{root_path}/dataTransformed.csv')

